# Running Auto Correlation with Talon Deployer and BITE
###### Last Updated: 19/03/24

This demo will show the basic operation of auto correlation using the new Deployer and BITE TANGO devices currently deployed in this repository. With this notebook, all TANGO commands and attribute changes are made via a [TANGO DeviceProxy](https://pytango.readthedocs.io/en/stable/client_api/device_proxy.html) but the overall steps should be the same for using the JIVE interface or Taranta web interface.

## Pre-requisites


First, this notebook assumes you have a running environment launched from a pipeline, in particular it assumes you are running off one launched from the [SKA-mid-psi](https://gitlab.com/ska-telescope/ska-mid-psi) pipeline. Secondly, for ease of dev work, it also assumes you are using a virtual env. This notebook was made with Python 3.10 in mind.

Finally, make sure all requirements are installed via [poetry]().

### Running on Dev servers

Firstly, to make future commands a bit easier, we load the namespace name into a variable.

In [ ]:
NS = "ska-mid-cbf"


To run this locally, a few extra steps are required:
* Uncomment the [pv.yaml file in Engineering Console](https://gitlab.com/ska-telescope/ska-mid-cbf-engineering-console/-/blob/main/charts/ska-mid-cbf-engineering-console-umbrella/templates/pv.yaml). This will establish the persistant volume, without having to go through MCS.
* Run the required make commands in the CICD repo.
    * `make minkube-clean DRIVER=docker`- Clean out the old minikube if previously used.
    * `make all DRIVER=docker` - Set up the minkube using the docker driver.
    * `eval $(minikube docker-env)` - Set the docker enivronment vairable.
* Then run the make commands to set up in the EC repo.
    * `make oci-build-all` - Build latest images/requriments in the repo.
    * `make oci-image-build` - Check to make sure the latest image is built.
    * `make k8s-reinstall-chart` - Deploy these images to the minikube.

If you check the minkube pods, you should see the TANGO database pods, the itango console and the old bite and deployer pods running. If they are still starting, give them a few moments.

In [ ]:
!kubectl -n {NS} get pods

This should return something like: 
```
NAME                                         READY   STATUS    RESTARTS   AGE
databaseds-ds-tango-databaseds-test-0        1/1     Running   0          17m
databaseds-tangodb-tango-databaseds-test-0   1/1     Running   0          17m
ds-bite-bite-0                               1/1     Running   0          16m
ds-deployer-deployer-0                       1/1     Running   0          16m
ds-tangotest-test-0                          1/1     Running   0          16m
ska-tango-base-itango-console                1/1     Running   0          17m
```

### Running on PSI

In [82]:
!{sys.executable} -m pip list

/bin/bash: {sys.executable}: command not found



To Run on PSI, once running the launch step from the PSI pipeline, grab your booted namespace's name. You can run a check on the ns as well to make sure the pods are ready.

In [27]:
!kubectl get ns

NAME                                                  STATUS   AGE
ci-ska-mid-cbf-sys-tests-1218202973-jkwso             Active   18h
ci-ska-mid-cbf-sys-tests-1218202973-jkwso-sdp         Active   18h
ci-ska-mid-cbf-sys-tests-1218364078-owendouglas       Active   18h
ci-ska-mid-cbf-sys-tests-1218364078-owendouglas-sdp   Active   18h
ci-ska-mid-cbf-sys-tests-1218390577-mparmis           Active   17h
ci-ska-mid-cbf-sys-tests-1218841021-epieters272       Active   8h
ci-ska-mid-cbf-sys-tests-1218841021-epieters272-sdp   Active   8h
ci-ska-mid-cbf-sys-tests-1219039994-epieters272       Active   6h4m
ci-ska-mid-cbf-sys-tests-1219039994-epieters272-sdp   Active   6h4m
ci-ska-mid-psi-1218359385-alexschell                  Active   50m
ci-ska-mid-psi-1218359385-alexschell-sdp              Active   50m
ci-ska-mid-psi-1218375439-amjoshi                     Active   17h
ci-ska-mid-psi-1218375439-amjoshi-sdp                 Active   17h
ci-ska-mid-psi-1218437587-maecst                      Active  

To make future work a bit easier we can load this into a NS var.

In [ ]:
# Change this var to be the non-spd namespace with your username in it
NS = "ci-ska-mid-psi-1218359385-alexschell"

We can then check to ensure the pods have spun up successfully.

In [ ]:
!kubectl -n $NS get pods

## Initial Setup/Using Deployer Device

First we want to set up the target board that we will be using for the auto correlation. While this can be multiple boards, for now we will only need one. Punch out the board(s) you have access to and set the `TARGET_BOARD` to assign it to be used in future steps.

The `TANGO_HOST` will be created based of the namespace you set earlier.

In [32]:
import os
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
# Set to the boards you wish to configure.
TARGET_BOARD = [3]


TANGO_HOST = "databaseds-tango-base."+NS+".svc.cluster.local:10000"
print("Will be using HOST: ", TANGO_HOST)

os.environ["TANGO_HOST"] = TANGO_HOST

Will be using HOST:  databaseds-tango-base.ci-ska-mid-psi-1218359385-alexschell.svc.cluster.local:10000


For this demo, we will interact with the TANGO devices via a device proxy, which will allow us to pass commands into them as we would in the UI.

In [48]:
from PyTango import DeviceProxy, Database

# Setup the device proxies targeting bite and deployer
db = Database()
deployer_tango = DeviceProxy("mid_csp_cbf/ec/deployer")

#Check the devices initially deployed to the database
devices = db.get_device_exported("*")
print(devices)
# Make sure the deployer device is set to ON
deployer_tango.On()
print("State of deployer device:")
print(deployer_tango.state())

DbDatum[
        name = '*'
value_string = ['dserver/CbfController/controller', 'dserver/CbfSubarray/cbfsubarray-01', 'dserver/CbfSubarray/cbfsubarray-02', 'dserver/CbfSubarray/cbfsubarray-03', 'dserver/DataBaseds/2', 'dserver/ECBite/bite', 'dserver/ECDeployer/deployer', 'dserver/FspMulti/fsp-01', 'dserver/FspMulti/fsp-02', 'dserver/FspMulti/fsp-03', 'dserver/FspMulti/fsp-04', 'dserver/MidCspCapabilityFsp/capabilityfsp', 'dserver/MidCspCapabilityVcc/capabilityvcc', 'dserver/MidCspController/controller', 'dserver/MidCspSubarray/subarray1', 'dserver/MidCspSubarray/subarray2', 'dserver/MidCspSubarray/subarray3', 'dserver/PowerSwitch/powerswitch-001', 'dserver/PowerSwitch/powerswitch-002', 'dserver/PowerSwitch/powerswitch-003', 'dserver/PowerSwitch/powerswitch-004', 'dserver/Slim/mesh', 'dserver/SlimLink/fs-links', 'dserver/SlimLink/vis-links', 'dserver/TalonBoard/talon-001', 'dserver/TalonBoard/talon-002', 'dserver/TalonBoard/talon-003', 'dserver/TalonBoard/talon-004', 'dserver/TalonBoard

First, we set the target talon boards we want to set up our configuration for. In Jive/Taranta, this would be configured by manually writing the attribute via the UI. Multiple boards can be targeted.

In [ ]:
deployer_tango.write_attribute("targetTalons",TARGET_BOARD)

With this set, we can then run the configuration command by calling the generate_config_jsons command.

In [ ]:
deployer_tango.command_inout("generate_config_jsons")

We then get the device artifacts from the [artifact repository](https://artefact.skao.int/#browse/browse:helm-internal) by running the command via TANGO. This step may take some time as it downloads multiple devices.

**! CURRENTLY THIS DOES NOT WORK ON MINIKUBE, MUST USE PSI !**

In [57]:
deployer_tango.set_timeout_millis(100000)
try:
    deployer_tango.command_inout("download_artifacts")
except Exception as e:
    print(e)
    print("Timed out, this is likely due to the download taking some time. Check the logs with the code space below after some time to see if it passes.")
deployer_tango.set_timeout_millis(4500)

DevFailed[
DevError[
    desc = TRANSIENT CORBA system exception: TRANSIENT_CallTimedout
  origin = Connection::command_inout()
  reason = API_CorbaException
severity = ERR]

DevError[
    desc = Timeout (100000 mS) exceeded on device mid_csp_cbf/ec/deployer, command download_artifacts
  origin = Connection::command_inout()
  reason = API_DeviceTimedOut
severity = ERR]
]
Timed out, this is likely due to the download taking some time. Check the logs with the code space below after some time to see if it passes.


To check that the artifacts downloaded successfully, we want to check that the following returns something like `INFO|Dummy-2|download_fpga_bitstreams|midcbf_deployer.py#418||Finished downloading`. 

In [58]:
!kubectl logs -n $NS ds-deployer-deployer-0 | grep 'Finished downloading'

1|2024-03-19T16:37:14.349Z|INFO|Dummy-2|download_fpga_bitstreams|midcbf_deployer.py#418||Finished downloading
1|2024-03-19T18:20:41.755Z|INFO|Dummy-3|download_fpga_bitstreams|midcbf_deployer.py#418||Finished downloading
1|2024-03-19T18:27:54.545Z|INFO|Dummy-3|download_fpga_bitstreams|midcbf_deployer.py#418||Finished downloading
1|2024-03-19T18:35:41.257Z|INFO|Dummy-3|download_fpga_bitstreams|midcbf_deployer.py#418||Finished downloading
1|2024-03-19T18:37:48.064Z|INFO|Dummy-1|download_fpga_bitstreams|midcbf_deployer.py#418||Finished downloading
1|2024-03-19T18:43:32.946Z|INFO|Dummy-2|download_fpga_bitstreams|midcbf_deployer.py#418||Finished downloading
1|2024-03-19T18:46:20.958Z|INFO|Dummy-2|download_fpga_bitstreams|midcbf_deployer.py#418||Finished downloading
1|2024-03-19T18:48:35.953Z|INFO|Dummy-1|download_fpga_bitstreams|midcbf_deployer.py#418||Finished downloading
1|2024-03-19T18:50:57.757Z|INFO|Dummy-3|download_fpga_bitstreams|midcbf_deployer.py#418||Finished downloading


Finally, we can configure the TANGO database with all the tango devices we just downloaded using the ConfigDB command.

In [61]:
deployer_tango.command_inout("configure_db")

The TANGO database should now be configured with all the devices needed for the next step, running the BITE device.

In [62]:
print(*db.get_device_exported("*").value_string, sep="\n")

dserver/CbfController/controller
dserver/CbfSubarray/cbfsubarray-01
dserver/CbfSubarray/cbfsubarray-02
dserver/CbfSubarray/cbfsubarray-03
dserver/DataBaseds/2
dserver/ECBite/bite
dserver/ECDeployer/deployer
dserver/FspMulti/fsp-01
dserver/FspMulti/fsp-02
dserver/FspMulti/fsp-03
dserver/FspMulti/fsp-04
dserver/MidCspCapabilityFsp/capabilityfsp
dserver/MidCspCapabilityVcc/capabilityvcc
dserver/MidCspController/controller
dserver/MidCspSubarray/subarray1
dserver/MidCspSubarray/subarray2
dserver/MidCspSubarray/subarray3
dserver/PowerSwitch/powerswitch-001
dserver/PowerSwitch/powerswitch-002
dserver/PowerSwitch/powerswitch-003
dserver/PowerSwitch/powerswitch-004
dserver/Slim/mesh
dserver/SlimLink/fs-links
dserver/SlimLink/vis-links
dserver/TalonBoard/talon-001
dserver/TalonBoard/talon-002
dserver/TalonBoard/talon-003
dserver/TalonBoard/talon-004
dserver/TalonBoard/talon-005
dserver/TalonBoard/talon-006
dserver/TalonBoard/talon-007
dserver/TalonBoard/talon-008
dserver/TalonDxLogConsumer/001


## Setting Controller settings

Now that we have the controller device deployed, we can use it to prep for further steps.

In [75]:
CBF_tango = DeviceProxy("mid_csp_cbf/sub_elt/controller")
print(CBF_tango.status())

The device is in OFF state.


We can then set adminMode to 0 (ONLINE), allowing us to run commands, and set simulationMode to 0 (FALSE).

In [83]:
CBF_tango.write_attribute("adminMode",0)
CBF_tango.write_attribute("simulationMode",0)
if CBF_tango.read_attribute("adminMode").value == 0 and CBF_tango.read_attribute("simulationmode").value == 0:
    print ("Set values successfully!")
else:
    print ("Error, couldn't set values!")

Set values successfully


Next, we load in a inital values parameters and pass it to the controller, to do this we read in JSON file and pass it as a DevString to the relevant device command:

In [95]:
# Change this value if you want to read files from somewhere else
FILE_LOCATION = os.getcwd()+"/json_files"

init_sys_param_file = os.path.join(FILE_LOCATION, "initial_system_param.json")

with open(init_sys_param_file) as init_file:
    data = init_file.read()

# Use the InitSysParam command and feed in data
CBF_tango.command_inout("InitSysParam", data)

[array([0], dtype=int32), ['CbfController InitSysParam command completed OK']]

## Running Commands Through the BITE Device

Now that the BITE tango Device has been deployed via the deployer, we can use it to configure tests. First we check the device is running.

In [ ]:
bite_tango = DeviceProxy("mid_csp_cbf/ec/bite")
# Running this should return RUNNING
print(bite_tango.State())

Now that we have confirmed it is running, we load in what board we want to configure.

In [34]:
bite_tango.write_attribute("boards",TARGET_BOARD)

For now, we can use the defaults and simply call the write command for the test configs. This should return the configuration for each board passed in.

In [ ]:
print(bite_tango.command_inout("generate_bite_data"))

Next, we actully start the LSTV replay.